## Neighbourhood battel for Borough in Queen in NYC

## Import all the libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          82 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge


geopy-1.17.0         | 49 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  envi

## Download New York City Data from IBM BOX.

In [2]:
!wget -q -O 'newyork_data.json' https://ibm.box.com/shared/static/fbpwbovar7lf8p5sgddm06cgipa2rxpe.json
print('Data downloaded!')

Data downloaded!


In [3]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
neighborhoods_data = newyork_data['features']

## Check Structure of the data

In [5]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

## Define cloumn for data frame

In [6]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [7]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


## Fetch data from Downloaded data frame

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


## Check Different Borough

In [11]:
print(neighborhoods['Borough'].unique())

['Bronx' 'Manhattan' 'Brooklyn' 'Queens' 'Staten Island']


## Fetch data for Borough Queens

In [12]:
queen_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queen_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


## Get geographical Coordinate for Queen, NY

In [13]:
address = 'Queens, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Queens are 40.6524927, -73.7914214158161.


## Create Folium map of Borough Queeen with neighborhoods superimposed on top

In [14]:
map_queen = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queen_data['Latitude'], queen_data['Longitude'], queen_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queen)  
    
map_queen

## Define Foursquare Credentials and Version

In [15]:
CLIENT_ID = 'TYFAOW3ZU2PS5LK3LLQTDZDLOFGZPKVXSHCR2IXFRYI12DAQ' # your Foursquare ID
CLIENT_SECRET = 'QPVHX4SI1JNW3HFHZWEO2YQRGFWMJ2H4L5DTJS53C24V04WX' # your Foursquare Secret
VERSION = '20181120' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TYFAOW3ZU2PS5LK3LLQTDZDLOFGZPKVXSHCR2IXFRYI12DAQ
CLIENT_SECRET:QPVHX4SI1JNW3HFHZWEO2YQRGFWMJ2H4L5DTJS53C24V04WX


## Get the neighborhood's latitude and longitude values.

In [16]:
neighborhood_latitude = queen_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = queen_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = queen_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Astoria are 40.76850859335492, -73.91565374304234.


## Create the GET request URL

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=TYFAOW3ZU2PS5LK3LLQTDZDLOFGZPKVXSHCR2IXFRYI12DAQ&client_secret=QPVHX4SI1JNW3HFHZWEO2YQRGFWMJ2H4L5DTJS53C24V04WX&v=20181120&ll=40.76850859335492,-73.91565374304234&radius=500&limit=100'

In [18]:
results = requests.get(url).json()

## get_category_type function from the Foursquare lab

In [19]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Fetch the Venue & Category data from JSON to Dataframe

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Favela Grill,Brazilian Restaurant,40.767348,-73.917897
1,Orange Blossom,Gourmet Shop,40.769856,-73.917012
2,Titan Foods Inc.,Gourmet Shop,40.769198,-73.919253
3,El Rey Del Taco Truck,Food Truck,40.767697,-73.918814
4,Off The Hook,Seafood Restaurant,40.767200,-73.918104


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## Explore Neighborhoods in Queen Borough by Get Near By Venue

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Create Queen_Venues for explore

In [23]:
queen_venues = getNearbyVenues(names=queen_data['Neighborhood'],
                                   latitudes=queen_data['Latitude'],
                                   longitudes=queen_data['Longitude']
                                  )

In [24]:
queen_venues.head(3)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop


## Drop Venue and Neighborhood Latitude and Longitude

In [27]:
venue_check = queen_venues.drop(['Neighborhood Latitude', 'Neighborhood Longitude', 'Venue Latitude', 'Venue Longitude'], axis=1)
venue_check.head(3)

,Neighborhood,Venue,Venue Category
0,Astoria,Favela Grill,Brazilian Restaurant
1,Astoria,Orange Blossom,Gourmet Shop
2,Astoria,Titan Foods Inc.,Gourmet Shop


## Group Venue and Category by Neighborhood

In [28]:
to_venue_by_boro = venue_check.groupby('Neighborhood', as_index=False)['Venue','Venue Category'].agg(lambda x: len(x.value_counts()))
to_venue_by_boro.sort_values(by='Neighborhood').head(3)

,Neighborhood,Venue,Venue Category
0,Arverne,15,11
1,Astoria,100,56
2,Astoria Heights,11,11


## Borough with most categories:

In [29]:
i_max = to_venue_by_boro['Venue Category'].idxmax()
print('The neighborhood with most different categories: ',to_venue_by_boro['Neighborhood'].iloc[i_max])

The neighborhood with most different categories:  Sunnyside Gardens


In [30]:
print(i_max)

76


In [31]:
to_venue_by_cat = venue_check.groupby('Venue Category', as_index=False)['Neighborhood'].agg(lambda x: len(x.value_counts()))
to_venue_by_cat.sort_values(by='Venue Category').head(3)

,Venue Category,Neighborhood
0,Accessories Store,2
1,Afghan Restaurant,2
2,American Restaurant,11


In [32]:
i_max = to_venue_by_cat['Neighborhood'].idxmax()
print('The most common category: ',to_venue_by_cat['Venue Category'].iloc[i_max])
print(i_max)

The most common category:  Deli / Bodega
69


## The most common venue category in boroughs:

In [34]:
to_venue_by_venue = venue_check.groupby('Venue', as_index=False)['Neighborhood', 'Venue Category'].agg(lambda x: len(x.value_counts()))
to_venue_by_venue.sort_values(by='Venue').head(3)

,Venue,Neighborhood,Venue Category
0,10 Points KTV,1,1
1,101 Café,1,1
2,101 Deli,1,1


In [35]:
i_max = to_venue_by_venue['Neighborhood'].idxmax()
print('The most common venue name: ', to_venue_by_venue['Venue'].iloc[i_max])
print(i_max)

The most common venue name:  Dunkin' Donuts
407


In [36]:
# one hot encoding
queen_onehot = pd.get_dummies(queen_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queen_onehot['Neighborhood'] = queen_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [queen_onehot.columns[-1]] + list(queen_onehot.columns[:-1])
queen_onehot = queen_onehot[fixed_columns]

queen_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [37]:
queen_grouped = queen_onehot.groupby('Neighborhood').mean().reset_index()
queen_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Bistro,Board Shop,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lighthouse,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,New American Restaurant,Nightclub,Noodle House,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Pool Hall,Post Office,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Club,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Arverne,0.000000,0.000000,0.000000,0.0

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

## Display the top 10 venues for each neighborhood.

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queen_grouped['Neighborhood']

for ind in np.arange(queen_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queen_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Metro Station,Board Shop,Beach,Thai Restaurant,Bed & Breakfast,Donut Shop,Sandwich Place,Bus Stop,Playground
1,Astoria,Middle Eastern Restaurant,Bar,Bakery,Greek Restaurant,Hookah Bar,Seafood Restaurant,Mediterranean Restaurant,Pub,Latin American Restaurant,Italian Restaurant
2,Astoria Heights,Hostel,Italian Restaurant,Pizza Place,Deli / Bodega,Bus Station,Burger Joint,Plaza,Bowling Alley,Supermarket,Bakery
3,Auburndale,Hookah Bar,Discount Store,Fast Food Restaurant,Miscellaneous Shop,Mobile Phone Shop,Furniture / Home Store,Supermarket,Korean Restaurant,Noodle House,Donut Shop
4,Bay Terrace,Clothing Store,Women's Store,Kids Store,Cosmetics Shop,Donut Shop,Mobile Phone Shop,Lingerie Store,Deli / Bodega,Gluten-free Restaurant,Men's Store
5,Bayside,Bar,Pizza Place,American Restaurant,Indian Restaurant,Donut Shop,Greek Restaurant,Italian Restaurant,Bakery,Mexican Restaurant,Sushi Restaurant
6,Bayswater,Playground,Park,Women's Store,Farmers Market,Event Space,Eye Doctor,Fabric Shop,Falafel Restaurant,Farm,Fast Food Restaurant
7,Beechhurst,Gym,Italian Restaurant,Deli / Bodega,Convenience Store,Pizza Place,Spa,Dessert Shop,Chinese Restaurant,Boutique,Shopping Mall
8,Bellaire,Deli / Bodega,Chinese Restaurant,Greek Restaurant,Bus Stop,Coffee Shop,Halal Restaurant,Moving Target,Italian Restaurant,Construction & Landscaping,Convenience Store
9,Belle Harbor,Beach,Deli / Bodega,Spa,Boutique,Italian Restaurant,Chinese Restaurant,Bakery,Bagel Shop,Donut Shop,Mexican Restaurant


## Select the food category and resturant

In [40]:
list_food = ['Restaurant', 'Cafe', 'Coffee', 'Food', 'Bar', 'Pub', 'Bakery', 'Beer', 'Ice', 'Burger', 'Diner', 'Pizza', 'Steak', 'Sushi', 'Sandwich', 'Mexican', 'Italian', 'Breakfast']

In [41]:
df_food = queen_venues

In [42]:
df_food.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,El Rey Del Taco Truck,40.767697,-73.918814,Food Truck
4,Astoria,40.768509,-73.915654,Off The Hook,40.767200,-73.918104,Seafood Restaurant
5,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
6,Astoria,40.768509,-73.915654,Simply Fit Astoria,40.769114,-73.912403,Gym
7,Astoria,40.768509,-73.915654,Irish Rover,40.765835,-73.914647,Pub
8,Astoria,40.768509,-73.915654,Sitan Muay Thai,40.766108,-73.913224,Martial Arts Dojo
9,Astoria,40.768509,-73.915654,SVL Bar,40.769816,-73.918530,Greek Restaurant


In [43]:
df_food = df_food[df_food['Venue Category'].isin(list_food)]
df_food.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
7,Astoria,40.768509,-73.915654,Irish Rover,40.765835,-73.914647,Pub
12,Astoria,40.768509,-73.915654,Neptune Diner,40.770788,-73.916890,Diner
19,Astoria,40.768509,-73.915654,Leli's Bakery and Pastry Shop,40.765132,-73.917696,Bakery
25,Astoria,40.768509,-73.915654,Sweet Afton,40.765350,-73.918983,Bar
31,Astoria,40.768509,-73.915654,Chip,40.765588,-73.919144,Bakery
36,Astoria,40.768509,-73.915654,The Shady Lady,40.765441,-73.918160,Bar
43,Astoria,40.768509,-73.915654,The Sparrow Tavern,40.772904,-73.916512,Bar
46,Astoria,40.768509,-73.915654,The Astorian,40.767314,-73.921210,Pub
53,Astoria,40.768509,-73.915654,International Meat Market,40.764840,-73.917000,Food
54,Astoria,40.768509,-73.915654,New York Bakery Café,40.766443,-73.920341,Bakery


## 4. Cluster Neighborhoods

# Run k-means to cluster the neighborhood into 5 clusters.

In [44]:
# set number of clusters
kclusters = 5

df_food_grouped = df_food.groupby('Neighborhood', as_index=False)['Venue Category'].count()
queen_grouped_clustering = df_food_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(queen_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 4, 3, 4, 4, 4, 0, 2, 2], dtype=int32)

In [45]:
queen_data.shape
queen_data.head(2)

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842


In [46]:
df_food_grouped.head(2)

,Neighborhood,Venue Category
0,Astoria,16
1,Astoria Heights,1


In [47]:
kmeans.labels_.shape

(54,)

In [48]:
queen_merged = df_food_grouped

# add clustering labels
queen_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
queen_merged = queen_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queen_merged.head() # check the last columns!

,Neighborhood,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,16,1,Middle Eastern Restaurant,Bar,Bakery,Greek Restaurant,Hookah Bar,Seafood Restaurant,Mediterranean Restaurant,Pub,Latin American Restaurant,Italian Restaurant
1,Astoria Heights,1,0,Hostel,Italian Restaurant,Pizza Place,Deli / Bodega,Bus Station,Burger Joint,Plaza,Bowling Alley,Supermarket,Bakery
2,Bay Terrace,3,4,Clothing Store,Women's Store,Kids Store,Cosmetics Shop,Donut Shop,Mobile Phone Shop,Lingerie Store,Deli / Bodega,Gluten-free Restaurant,Men's Store
3,Bayside,9,3,Bar,Pizza Place,American Restaurant,Indian Restaurant,Donut Shop,Greek Restaurant,Italian Restaurant,Bakery,Mexican Restaurant,Sushi Restaurant
4,Belle Harbor,2,4,Beach,Deli / Bodega,Spa,Boutique,Italian Restaurant,Chinese Restaurant,Bakery,Bagel Shop,Donut Shop,Mexican Restaurant


In [49]:
queen_merged.shape

(54, 13)

In [50]:
queen_merged_1 = queen_merged.join(queen_data.set_index('Neighborhood'), on='Neighborhood')

In [51]:
queen_merged_1.shape

(54, 16)

In [52]:
queen_merged.head(3)

,Neighborhood,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,16,1,Middle Eastern Restaurant,Bar,Bakery,Greek Restaurant,Hookah Bar,Seafood Restaurant,Mediterranean Restaurant,Pub,Latin American Restaurant,Italian Restaurant
1,Astoria Heights,1,0,Hostel,Italian Restaurant,Pizza Place,Deli / Bodega,Bus Station,Burger Joint,Plaza,Bowling Alley,Supermarket,Bakery
2,Bay Terrace,3,4,Clothing Store,Women's Store,Kids Store,Cosmetics Shop,Donut Shop,Mobile Phone Shop,Lingerie Store,Deli / Bodega,Gluten-free Restaurant,Men's Store


In [53]:
queen_merged_1.head(3)

,Neighborhood,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Borough,Latitude,Longitude
0,Astoria,16,1,Middle Eastern Restaurant,Bar,Bakery,Greek Restaurant,Hookah Bar,Seafood Restaurant,Mediterranean Restaurant,Pub,Latin American Restaurant,Italian Restaurant,Queens,40.768509,-73.915654
1,Astoria Heights,1,0,Hostel,Italian Restaurant,Pizza Place,Deli / Bodega,Bus Station,Burger Joint,Plaza,Bowling Alley,Supermarket,Bakery,Queens,40.770317,-73.894680
2,Bay Terrace,3,4,Clothing Store,Women's Store,Kids Store,Cosmetics Shop,Donut Shop,Mobile Phone Shop,Lingerie Store,Deli / Bodega,Gluten-free Restaurant,Men's Store,Queens,40.782843,-73.776802


## Create Folium map for visulization

In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queen_merged_1['Latitude'], queen_merged_1['Longitude'], queen_merged_1['Neighborhood'], queen_merged_1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters